<a href="https://colab.research.google.com/github/rohank36/rohanBot/blob/main/RohanBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradientai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.6/353.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 42.8 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.6.4
    Uninstalling pydantic-2.6.4:
      Successfully uninstalled pydantic-2.6.4


In [65]:
import os
os.environ['GRADIENT_ACCESS_TOKEN'] = ""
os.environ['GRADIENT_WORKSPACE_ID'] = ""

In [ ]:
import csv

from gradientai import Gradient

path_to_csv_file = "/content/rohanBot_dataset.csv"

In [ ]:
print("Parsing data...")
rows_to_keep = []
with open(path_to_csv_file, encoding="utf-8-sig") as f:
  reader = csv.DictReader(f, delimiter=",")
  last_row = None
  for row in reader:
    if "Rohan" == row["name"] and last_row is not None:
      rows_to_keep.append(last_row)
      rows_to_keep.append(row)
      last_row = None
    else:
      last_row = row

role_play_prompt = "You are Rohan Kanti, a 22 year old ex-Singapore Army Liuetenant, computer science and finance student who loves Brazilian Jiu Jitsu. You are easy going, kind, funny, determined. You always look to make good banter when chatting with your friends. Respond to the following line of dialog as Rohan Kanti."

lines = []
for i in range(0, len(rows_to_keep), 2):
  prompt = rows_to_keep[i]
  response = rows_to_keep[i+1]
  start_str = f"<s>### Instruction:\n{role_play_prompt}\n\n###Input:\n"
  prompt = prompt["line"].replace('"','\\"')
  mid_str = '''\n\n### Response:\n'''
  response = response["line"].replace('"','\\"')
  end_str = '''</s>'''
  total_line = start_str + prompt + mid_str + response + end_str
  obj = {
      "inputs" : total_line
  }
  lines.append(obj)

print(f"Generated {len(lines)} lines to fine-tune")
print(f"Example training line: {lines[0]}")

lines_per_chunk = 20
all_chunks = []
for line in lines:
  if len(all_chunks) == 0 or len(all_chunks[-1]) == lines_per_chunk:
    all_chunks.append([])
  all_chunks[-1].append(line)

print(f"\nFine-tuning model adapter")
gradient = Gradient()
base = gradient.get_base_model(base_model_slug="nous-hermes2")
my_adapter = base.create_model_adapter(name="rohanbot")
print(f"Created model with ID {my_adapter.id}")
for i in range(len(all_chunks)):
  try:
    print(f"Fine-tuning chunk {i} of {len(all_chunks) - 1}")
    my_adapter.fine_tune(samples=all_chunks[i])
  except Exception as error:
    try:
      error_pieces = str(error).split("\n")
      if len(error_pieces) > 1:
        print(f"*** Error processing chunk {i}: {error_pieces[0]} {error_pieces[1]}")
      else:
        print(f"*** Unknown error on chunk {i}: {error}")
    except KeyboardInterrupt:
      break
    except Exception as inner_error:
      print(inner_error)


Parsing data...
Generated 151 lines to fine-tune
Example training line: {'inputs': "<s>### Instruction:\nYou are Rohan Kanti, a 22 year old ex-Singapore Army Liuetenant, computer science and finance student who loves Brazilian Jiu Jitsu. You are easy going, kind, funny, determined. You always look to make good banter when chatting with your friends. Respond to the following line of dialog as Rohan Kanti.\n\n###Input:\nlook how live the lower field is rn \n\n### Response:\nIKR my sister is there too, must be so fun it's so nice outside today</s>"}

Fine-tuning model adapter
Created model with ID 681f75f8-a458-4019-baf7-cb2080a6696f_model_adapter
Fine-tuning chunk 0 of 7
Fine-tuning chunk 1 of 7
Fine-tuning chunk 2 of 7
Fine-tuning chunk 3 of 7
Fine-tuning chunk 4 of 7
Fine-tuning chunk 5 of 7
Fine-tuning chunk 6 of 7
Fine-tuning chunk 7 of 7


In [ ]:
# if your colab instance gets deleted, you can run this to get the model names
gradient = Gradient()
# if necessary, go back and find your previously created models and their IDs
old_models = gradient.list_models(only_base=False)
for model in old_models:
  if hasattr(model, "name"):
    print(f"{model.name}: {model.id}")

In [64]:
role_play_prompt = "You are Rohan Kanti, a 22 year old ex-Singapore Army Liuetenant, computer science and finance student who loves Brazilian Jiu Jitsu. You are easy going, kind, funny, determined. Respond to the following line of dialog as Rohan Kanti."
query = "Amazing lets do it, are you in?"
templated_query = f"<s>### Instruction:\n{role_play_prompt}\n\n###Input:\n{query}\n\n### Response:\n"
response = my_adapter.complete(query=templated_query, max_generated_token_count=500)
print(f"> {query}\n> {response.generated_output}\n\n")


> Amazing lets do it, are you in?
> Yeah I'm in, I'll see you there




In [ ]:
my_adapter.delete()